# Lab 2 Data engineering and wrangling
1. Read in dataset
2. Clean / encode dataset 
3. Save dataset back into project repo

In [ ]:
import numpy as np
import pandas as pd
import os
import json
import seaborn as sns
sns.set(font_scale=1.5)

%matplotlib inline 

# Loading student variable
%store -r STUDENT

In [ ]:
def ProjectRepo(path):
   ProjectRepo = "/bd-fs-mnt/project_repo"
   return str(ProjectRepo + '/' + path)

# Let's view our dataset to see what it looks like

In [ ]:
train_file = ProjectRepo('data/' + STUDENT + '_UCI_Income/adult_data.csv')
test_file = ProjectRepo('data/' + STUDENT + '_UCI_Income/adult_test.csv')

In [ ]:
train_set = pd.read_csv(train_file, header=None)
train_set.head()

In [ ]:
test_set = pd.read_csv(test_file, skiprows=1, header=None)
test_set.head()

## Initial Findings
1. No column headers (can fix using dataset description from website)
2. Some "?" in test data 
3. Target values differ in train and test set

#### 1. Fix column headers

In [ ]:
col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
             'wage_class']

In [ ]:
train_set.columns = col_labels
test_set.columns = col_labels

#### 2. Clean up ? in data

In [ ]:
train_set.info()

In [ ]:
test_set.info()

In [ ]:
train_set.replace(' ?', np.nan).dropna().shape

In [ ]:
test_set.replace(' ?', np.nan).dropna().shape

In [ ]:
# removing rows with "?" from our dataframes 
train_no_missing = train_set.replace(' ?', np.nan).dropna()
test_no_missing = test_set.replace(' ?', np.nan).dropna()

#### 3. Fix targets (remove the extra periods from '<=50K.' to '<=50K')

In [ ]:
test_no_missing['wage_class'] = test_no_missing.wage_class.replace({' <=50K.' : ' <=50K', ' >50K.' : ' >50K'})

In [ ]:
test_no_missing.wage_class.unique()

In [ ]:
train_no_missing.wage_class.unique()

## Applying ordinal encoding to categoricals
- ordinal encoding: convert string labels to integer values 1 through k. First unique value in column becomes 1, the second becomes 2, the third becomes 3, and so on


In [ ]:
#combine the datasets together first
combined_set = pd.concat([train_no_missing, test_no_missing], axis=0)
combined_set.info()

In [ ]:
#Visualizations after initial cleaning of dataset 
group = combined_set.groupby('wage_class')
group

In [ ]:
#encode non-numerical features into numeric values using pandas Cateogrical codes 
#and generating categorical codes mapping into dictionary
cat_codes = {}
for feature in combined_set.columns: 
    if combined_set[feature].dtype == 'object':
        #workclass : { occupation : number }
        temp_dict = {}
        feature_codes = list(pd.Categorical(combined_set[feature]).codes)
        feature_list = list(combined_set[feature])
        for i in range(len(feature_codes)):
            temp_dict[feature_list[i].strip()] = int(feature_codes[i])
            if len(temp_dict) > len(feature_list):
                break
        cat_codes[feature] = temp_dict
        combined_set[feature] = pd.Categorical(combined_set[feature]).codes
combined_set.info()

In [ ]:
# saving encoding to json file to be used for scoring script
json_file = ProjectRepo('data/' + STUDENT + '_UCI_Income/encoding.json')
with open(json_file, 'w') as file:
    json.dump(cat_codes, file)

In [ ]:
#split combined set back into test/train split 
final_train = combined_set[:train_no_missing.shape[0]] 
final_test = combined_set[train_no_missing.shape[0]:]
final_train.to_csv(ProjectRepo('data/' + STUDENT + '_UCI_Income/adult_train_cleaned.csv'))
final_test.to_csv(ProjectRepo('data/' + STUDENT + '_UCI_Income/adult_test_cleaned.csv'))

# Continue onto Lab 3 for modeling! 